In [1]:
import numpy as np
from pyscf import ao2mo, gto, scf

from tn4qa.dmrg import DMRG
from tn4qa.qi_metrics import get_two_orbital_rdm

####
# First we need to generate some test data
####

# Define H2 molecule
mol = gto.Mole()
mol.atom = "H 0 0 0; H 0 0 0.74"
mol.basis = "sto-3g"
mol.spin = 0
mol.build()

# RHF calculation
mf = scf.RHF(mol)
mf.kernel()

# One-electron integrals
hcore = mf.get_hcore()
norb = hcore.shape[0]

# Convert to spin-orbital basis
nso = 2 * norb
h1 = np.kron(np.eye(2), hcore)

# Two-electron integrals (spatial orbital basis)
eri = mol.intor("int2e")  # (ij|kl)
eri_spin = ao2mo.restore(1, eri, norb)  # 4-index in chemist's notation

# Convert to spin-orbital integrals
g = np.zeros((nso, nso, nso, nso))

# Fill spin-orbital 2e integrals
for p in range(norb):
    for q in range(norb):
        for r in range(norb):
            for s in range(norb):
                value = eri_spin[p, q, r, s]
                # alpha-alpha block
                g[2 * p, 2 * q, 2 * r, 2 * s] = value
                # beta-beta block
                g[2 * p + 1, 2 * q + 1, 2 * r + 1, 2 * s + 1] = value

print(h1.shape, g.shape)

converged SCF energy = -1.11675930739642
(4, 4) (4, 4, 4, 4)


In [2]:
####
# Next we run a fermionic DMRG calculation
####

# Expected FCI energy is about -1.13
for _ in range(1):
    dmrg = DMRG(
        hamiltonian=(h1, g),
        max_mps_bond=32,
        method="two-site",
        hamiltonian_type="fermionic",
    )
    dmrg.run(maxiter=10)
    print(dmrg.energy)

-2.2419189125842314


In [3]:
####
# Now we can find the RDMs
####

rdm2 = get_two_orbital_rdm(dmrg.mps, [1, 2])

AttributeError: 'complex' object has no attribute 'tensor_to_matrix'